In [1]:
# mask v2 0.80463
import pandas as pd
import numpy as np
import gc
from base import Cache
from tqdm import tqdm
from gensim.models import Word2Vec
import sys
from tensorflow.keras.preprocessing.sequence import pad_sequences
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('precision', 5)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option('max_colwidth', 200)
pd.set_option('display.width', 5000)
# # log
# class Logger(object):
#     def __init__(self, fileN="Default.log"):
#         self.terminal = sys.stdout
#         self.log = open(fileN, "a", encoding='utf-8')
# 
#     def write(self, message):
#         self.terminal.write(message)
#         self.log.write(message)
# 
#     def flush(self):
#         pass
# sys.stdout = Logger("zlh0918log.txt")

def reduce_mem(df, use_float16=False):
    start_mem = df.memory_usage().sum() / 1024**2
    tm_cols = df.select_dtypes('datetime').columns
    for col in df.columns:
        if col in tm_cols:
            continue
        col_type = df[col].dtypes
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(
                        np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(
                        np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(
                        np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(
                        np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if use_float16 and c_min > np.finfo(
                        np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(
                        np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    print('{:.2f} Mb, {:.2f} Mb ({:.2f} %)'.format(
        start_mem, end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

def w2v_pro(df_raw, sentence_id, word_id, emb_size=128, 
            dropna=False, n_jobs=16, method='cbow', 
            hs=1,negative=0,epoch=10,return_model=True):
    if method.lower() in ['sg','skipgram']:
        sg=1
        logger.info("## Use skip-gram ##")
    elif method.lower() in ['cbow']:
        sg=0
        logger.info("## Use CBOW ##")
    else:
        raise NotImplementedError
    list_col_nm = f'{sentence_id}__{word_id}_list'
    if (n_jobs is None) or (n_jobs <= 0):
        n_jobs = multiprocessing.cpu_count()
    logger.info(f"========== W2V:  {sentence_id} {word_id} ==========")
    df = df_raw[[sentence_id, word_id]].copy()
    if df[sentence_id].isnull().sum() > 0:
        logger.warning("NaNs exist in sentence_id column!!")
    if dropna:
        df = df.dropna(subset=[sentence_id, word_id])
    else:
        df = df.fillna('NULL')
    df = df.astype(str)
    tmp = df.groupby(sentence_id,
                     as_index=False)[word_id].agg({list_col_nm: list})
    sentences = tmp[list_col_nm].values.tolist()
    all_words_vocabulary = df[word_id].unique().tolist()
    del tmp[list_col_nm]
    gc.collect()
    model = Word2Vec(
        sentences,
        size=emb_size,
        window=30,
        workers=n_jobs,
        min_count=1,  # 最低词频. min_count>1会出现OOV
        sg=sg,  # 1 for skip-gram; otherwise CBOW.
        hs=hs,  # If 1, hierarchical softmax will be used for model training
        negative=negative,  # hs=1 + negative 负采样
        iter=epoch,
        seed=0)
    
    # get word embedding matrix
    emb_dict = {}
    for word_i in all_words_vocabulary:
        if word_i in model.wv:
            emb_dict[word_i] = model.wv[word_i]
        else:
            emb_dict[word_i] = np.zeros(emb_size)
            
    return {"word_emb_dict": emb_dict}

def get_sequence(data,col,max_len=None):
    key2index = {}
    def split(x):
        for key in x:
            if key not in key2index:
                # Notice : input value 0 is a special "padding", 
                # so we do not use 0 to encode valid feature for sequence input
                key2index[key] = len(key2index) + 1 # 从1开始，0用于padding
        return list(map(lambda x: key2index[x], x))
    
    # preprocess the sequence feature
    id_list = list(map(split, data[col].values))# 转index
    id_list_length = np.array(list(map(len, id_list)))
    # max_len = max(genres_length)
    if max_len is None:
        max_len = int(np.percentile(id_list_length,99))
    id_list = pad_sequences(id_list, maxlen=max_len, padding='post',truncating='post')
    return id_list,key2index

def gen_list_df(feature):
    print(f'{feature} start!')
    data = Cache.reload_cache('CACHE_data_step_1_feature_0917_r5.pkl')
    if feature =='label':
        data['label'] = data['label'].fillna(2).astype(int)# mask 2
    data = data[['uid',feature,'pt_d']]
    gc.collect()
    print(data.shape)
    data_group = data.groupby(['uid'])
    gc.collect()
    index_list = []
    feature_list = []
    print('index_list start')
    for name,group in tqdm(data_group):
        index_list.append(name)    
    print('feature_list start')
    for i in tqdm(index_list):
        index_get_group = data_group.get_group(i)
        ptd_set = set(index_get_group['pt_d'].values.flatten().tolist())
        for j in ptd_set:
            feature_list_ = []
            buf_list = []
            buf_list = index_get_group.query('pt_d < @j')[feature].values.flatten().tolist()
            buf_list.append(2)# padding 0
            feature_list_.append(buf_list)# 行为序列
            feature_list_.append(j)# pt_d
            feature_list_.append(i)# uid
            feature_list.append(feature_list_)

    list_df = pd.DataFrame(feature_list)
    del index_list,feature_list,feature_list_,data_group,index_get_group,ptd_set
    gc.collect()
    list_df.columns=['list','pt_d','uid']
    list_df['list'] = list_df['list'].map(lambda x: [str(i) for i in x])# 转str
    list_df = list_df.drop_duplicates(subset=['pt_d','uid'])
#     data_uid_ptd = data[['uid','pt_d']]
    list_df = data.merge(list_df,how='left',on=('uid','pt_d'))# 顺序还是用data的顺序
    # 加入当天本样本
    if feature!='label':
        list_df['list'] = list_df[feature].map(lambda x:[str(x)]) + list_df['list']
    print('w2v start!')
    emb_size = 32
    model = Word2Vec(
    list_df['list'].values.tolist(),
    size=emb_size,
    window=5,
    workers=5,
    min_count=1,  # 最低词频. min_count>1会出现OOV
    sg=0,  # 1 for skip-gram; otherwise CBOW.
    hs=0,  # If 1, hierarchical softmax will be used for model training
    negative=5,  # hs=1 + negative 负采样
    iter=5,
    seed=0)
    # 1 获取seq
    id_list,key2index = get_sequence(list_df,'list',max_len=40)
    # 2 获取key2index
    emb_dict = {}
    for word_i in list(model.wv.vocab.keys()):
        if word_i in model.wv:
            emb_dict[word_i] = model.wv[word_i]
        else:
            emb_dict[word_i] = np.zeros(emb_size)
    # 3 保存
    id_list_dict={}
    id_list_dict['id_list'] = id_list
    id_list_dict['key2index'] = key2index
    id_list_dict['emb'] = emb_dict
    Cache.cache_data(id_list_dict, nm_marker=f'EMB_INPUTSEQ_V2_{feature}')
    print(f'{feature} done!')

from multiprocessing import Pool
if __name__ == '__main__':
    # 获取过去的list + 当前的一行
    # 得到id_list_dict和tx一样
    poc_feature_list = ['creat_type_cd','tags','spread_app_id','task_id','adv_id','label']#'task_id','adv_id','dev_id','inter_type_cd','spread_app_id','tags','app_first_class','app_second_class','his_app_size','his_on_shelf_time','app_score',,'creat_type_cd','adv_prim_id','indu_name'
    with Pool(6) as p:
        p.map(gen_list_df, poc_feature_list)


spread_app_id start!
label start!
task_id start!
creat_type_cd start!
adv_id start!
tags start!


[2020-09-19 20:07:41] - __init__.py[line:126] - INFO: Successfully Reload: /home/zhangqibot/proj/digix/zlh/cached_data/CACHE_data_step_1_feature_0917_r5.pkl
[2020-09-19 20:07:41] - __init__.py[line:126] - INFO: Successfully Reload: /home/zhangqibot/proj/digix/zlh/cached_data/CACHE_data_step_1_feature_0917_r5.pkl
[2020-09-19 20:07:41] - __init__.py[line:126] - INFO: Successfully Reload: /home/zhangqibot/proj/digix/zlh/cached_data/CACHE_data_step_1_feature_0917_r5.pkl
[2020-09-19 20:07:42] - __init__.py[line:126] - INFO: Successfully Reload: /home/zhangqibot/proj/digix/zlh/cached_data/CACHE_data_step_1_feature_0917_r5.pkl
[2020-09-19 20:07:42] - __init__.py[line:126] - INFO: Successfully Reload: /home/zhangqibot/proj/digix/zlh/cached_data/CACHE_data_step_1_feature_0917_r5.pkl
[2020-09-19 20:07:42] - __init__.py[line:126] - INFO: Successfully Reload: /home/zhangqibot/proj/digix/zlh/cached_data/CACHE_data_step_1_feature_0917_r5.pkl


(8601298, 3)
(8601298, 3)
index_list start
(8601298, 3)
index_list start
index_list start
(8601298, 3)
index_list start
(8601298, 3)
(8601298, 3)
index_list start
index_list start


100%|██████████| 1139171/1139171 [01:57<00:00, 9706.09it/s] 


feature_list start


...

[2020-09-19 21:43:50] - base_any2vec.py[line:1291] - INFO: EPOCH 5 - PROGRESS: at 44.73% examples, 1535476 words/s, in_qsize 9, out_qsize 0
[2020-09-19 21:43:50] - base_any2vec.py[line:1291] - INFO: EPOCH 5 - PROGRESS: at 27.58% examples, 1490422 words/s, in_qsize 8, out_qsize 1
[2020-09-19 21:43:51] - base_any2vec.py[line:1291] - INFO: EPOCH 5 - PROGRESS: at 46.76% examples, 1535731 words/s, in_qsize 9, out_qsize 0
[2020-09-19 21:43:51] - base_any2vec.py[line:1291] - INFO: EPOCH 5 - PROGRESS: at 29.67% examples, 1494679 words/s, in_qsize 9, out_qsize 0
[2020-09-19 21:43:52] - base_any2vec.py[line:1291] - INFO: EPOCH 5 - PROGRESS: at 48.92% examples, 1538054 words/s, in_qsize 10, out_qsize 0
[2020-09-19 21:43:52] - base_any2vec.py[line:1291] - INFO: EPOCH 5 - PROGRESS: at 31.65% examples, 1496914 words/s, in_qsize 9, out_qsize 0
[2020-09-19 21:43:53] - base_any2vec.py[line:1291] - INFO: EPOCH 5 - PROGRESS: at 50.92% examples, 1536958 words/s, in_qsize 9, out_qsize 0
[2020-09-19 21:43:5

[2020-09-19 21:44:17] - base_any2vec.py[line:1332] - INFO: EPOCH - 5 : training on 92062788 raw words (76143050 effective words) took 49.7s, 1532163 effective words/s
[2020-09-19 21:44:17] - base_any2vec.py[line:1368] - INFO: training on a 460313940 raw words (380709836 effective words) took 251.2s, 1515541 effective words/s
[2020-09-19 21:44:18] - base_any2vec.py[line:1291] - INFO: EPOCH 5 - PROGRESS: at 82.10% examples, 1517474 words/s, in_qsize 9, out_qsize 0
[2020-09-19 21:44:19] - base_any2vec.py[line:1291] - INFO: EPOCH 5 - PROGRESS: at 84.20% examples, 1520910 words/s, in_qsize 8, out_qsize 1
[2020-09-19 21:44:20] - base_any2vec.py[line:1291] - INFO: EPOCH 5 - PROGRESS: at 86.26% examples, 1521954 words/s, in_qsize 9, out_qsize 0
[2020-09-19 21:44:21] - base_any2vec.py[line:1291] - INFO: EPOCH 5 - PROGRESS: at 88.24% examples, 1521174 words/s, in_qsize 9, out_qsize 0
[2020-09-19 21:44:22] - base_any2vec.py[line:1291] - INFO: EPOCH 5 - PROGRESS: at 90.28% examples, 1522435 words/

adv_id done!


[2020-09-19 21:45:21] - __init__.py[line:111] - INFO: Cache Successfully! File name: /home/zhangqibot/proj/digix/zlh/cached_data/CACHE_EMB_INPUTSEQ_V2_task_id.pkl


task_id done!


In [2]:
import pandas as pd
import numpy as np
import gc
from base import Cache
from tqdm import tqdm
from gensim.models import Word2Vec
data = Cache.reload_cache('CACHE_data_step_1_feature_0917_r5.pkl')
seq_emb = Cache.reload_cache('CACHE_EMB_INPUTSEQ_V2_adv_id.pkl')

[2020-09-19 21:45:28] - __init__.py[line:126] - INFO: Successfully Reload: /home/zhangqibot/proj/digix/zlh/cached_data/CACHE_data_step_1_feature_0917_r5.pkl
[2020-09-19 21:45:30] - __init__.py[line:126] - INFO: Successfully Reload: /home/zhangqibot/proj/digix/zlh/cached_data/CACHE_EMB_INPUTSEQ_V2_adv_id.pkl


In [3]:
data[['index','uid','pt_d','adv_id']].head()

,index,uid,pt_d,adv_id
0,17073310,2237673,5,6340
1,17636486,2237673,5,4501
2,35175266,2237672,6,3701
3,30784519,2237672,4,5389
4,11939625,2237672,4,4506


In [4]:
data.query('uid==2237673')[['index','uid','pt_d','adv_id']]

,index,uid,pt_d,adv_id
0,17073310,2237673,5,6340
1,17636486,2237673,5,4501


In [5]:
seq_emb.keys()

dict_keys(['id_list', 'key2index', 'emb'])

In [6]:
seq_emb['id_list'][:5,:]

array([[1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [3, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [4, 5, 6, 7, 8, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [5, 7, 8, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [6, 7, 8, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int32)

In [7]:
seq_emb['key2index']['6340']

1

In [8]:
seq_emb['key2index']['4501']

3

### 检查过了，seq 和 dense 可以对上！